# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b IemocapClass https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 176 (delta 57), reused 47 (delta 25), pack-reused 94 (from 3)
Receiving objects: 100% (176/176), 62.25 KiB | 12.45 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
RAVDESS pronto in cache: /kaggle/input/ravdess-emotional-speech-audio
Numero totale di file: 2880

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: dejolilandry/iemocapfullrelease...
Using Colab cache for faster access to the 'iemocapfullrelease' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/iemocapfullrelease
IEMOCAP pronto in cache: /kaggle/input/iemocapfullrelease
Numero totale di file: 81249

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo
IEMOCAP: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [ ]:
import os
from pathlib import Path

# Check kagglehub cache directory
kaggle_cache_base = Path('/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease')

print("=" * 80)
print("KAGGLEHUB CACHE EXPLORATION")
print("=" * 80)

if kaggle_cache_base.exists():
    print(f"✓ Base cache directory exists: {kaggle_cache_base}\n")
    
    # Check versions subdirectory
    versions_dir = kaggle_cache_base / 'versions'
    if versions_dir.exists():
        print(f"✓ Versions directory exists\n")
        
        for version in sorted(versions_dir.iterdir()):
            if version.is_dir():
                print(f"📂 Version: {version.name}/")
                
                # Show IEMOCAP_full_release
                iemocap_dir = version / 'IEMOCAP_full_release'
                if iemocap_dir.exists():
                    print(f"  └─ 📂 IEMOCAP_full_release/")
                    
                    # Show Sessions
                    sessions = sorted([d for d in iemocap_dir.iterdir() if d.is_dir() and d.name.startswith('Session')])
                    print(f"    └─ Found {len(sessions)} sessions:")
                    
                    for session_dir in sessions[:2]:  # Show first 2 sessions
                        print(f"      📁 {session_dir.name}/")
                        
                        # Show subdirectories in session
                        subdirs = sorted([d.name for d in session_dir.iterdir() if d.is_dir()])
                        for subdir_name in subdirs[:5]:  # Show first 5 subdirs
                            print(f"        ├─ {subdir_name}/")
                        
                        if len(subdirs) > 5:
                            print(f"        └─ ... and {len(subdirs) - 5} more")
                    
                    if len(sessions) > 2:
                        print(f"      ... and {len(sessions) - 2} more sessions")
else:
    print(f"✗ Cache directory not found: {kaggle_cache_base}")

print("\n" + "=" * 80)


KAGGLEHUB CACHE EXPLORATION
✓ Base cache directory exists: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease

Contents of cache directory:
  1.complete
  versions/ 

✓ Versions directory exists: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions
Contents of versions/:
  1/ 
    Contents of 1/:
      IEMOCAP_full_release/ 



In [11]:
from torch.utils.data import DataLoader
from dataset.custom_iemocap_dataset import CustomIEMOCAPDataset
import os

# Detect environment and set dataset path automatically
if os.path.exists('/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1'):
    # Running with kagglehub cache (local or Colab with kagglehub)
    dataset_path = '/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1'
    print("✓ Found IEMOCAP in kagglehub cache")
elif os.path.exists('/kaggle/input/iemocapfullrelease'):
    # Running on Kaggle environment
    dataset_path = '/kaggle/input/iemocapfullrelease'
    print("✓ Detected Kaggle environment")
else:
    # Running locally or Colab
    dataset_path = './data/'
    print("✓ Using local data path")

print(f"📂 Using dataset path: {dataset_path}")

# Check if dataset exists
if not os.path.exists(dataset_path):
    print(f"⚠️ Warning: Dataset path not found: {dataset_path}")

# Create train dataset
print("\n🔄 Loading IEMOCAP dataset...")
train_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='train')
test_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='test')

print(f"\n✓ Train samples: {len(train_dataset)}")
print(f"✓ Test samples: {len(test_dataset)}")

# Create DataLoaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✓ DataLoaders created!")
print(f"  - Train batches: {len(train_dataloader)}")
print(f"  - Test batches: {len(test_dataloader)}")


✓ Found IEMOCAP in kagglehub cache
📂 Using dataset path: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1

🔄 Loading IEMOCAP dataset...
📂 Scanning dataset directory: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1

✅ Collected 0 samples in total


ValueError: No samples found in dataset!

In [ ]:
# Visualize training samples
from utils.visualization import visualize_samples, print_batch_info
print("Visualizing samples from training set...")
batch = next(iter(train_dataloader))
print_batch_info(batch)
print("\n")
visualize_samples(train_dataloader, num_samples=4)


ModuleNotFoundError: No module named 'utils.visualization'

# Step 4: Train your model and visualize training

In [ ]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [ ]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
#!python eval.py